# Using CSBM generated data to train a model

In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import math
from csbms import MultiClassCSBM, StructureCSBM

In [2]:
n = 1600
d = 128
c = 16

In [3]:
csbm = MultiClassCSBM(n=n, dimensions=d, classes=c)
data_list = []
for _ in range(10):
    data_list.append(csbm.data)
    csbm.evolve()

In [8]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

dataloader = DataLoader(data_list, batch_size=128)

In [10]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(d, 16)
        self.conv2 = GCNConv(16, c)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [11]:
num_epochs = 200
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

for epoch in range(num_epochs):
    model.train()
    for batch in dataloader:
        data = batch.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

cuda
Epoch 1/200, Loss: 2.878124713897705
Epoch 2/200, Loss: 2.817547559738159
Epoch 3/200, Loss: 2.7753052711486816
Epoch 4/200, Loss: 2.7290477752685547
Epoch 5/200, Loss: 2.703495740890503
Epoch 6/200, Loss: 2.6649389266967773
Epoch 7/200, Loss: 2.6480908393859863
Epoch 8/200, Loss: 2.6081387996673584
Epoch 9/200, Loss: 2.5774734020233154
Epoch 10/200, Loss: 2.5620970726013184
Epoch 11/200, Loss: 2.5242741107940674
Epoch 12/200, Loss: 2.512272834777832
Epoch 13/200, Loss: 2.465965986251831
Epoch 14/200, Loss: 2.4530630111694336
Epoch 15/200, Loss: 2.4190056324005127
Epoch 16/200, Loss: 2.395930290222168
Epoch 17/200, Loss: 2.372180223464966
Epoch 18/200, Loss: 2.340902805328369
Epoch 19/200, Loss: 2.3169431686401367
Epoch 20/200, Loss: 2.2962756156921387
Epoch 21/200, Loss: 2.2787466049194336
Epoch 22/200, Loss: 2.258518934249878
Epoch 23/200, Loss: 2.2175323963165283
Epoch 24/200, Loss: 2.194798231124878
Epoch 25/200, Loss: 2.1823549270629883
Epoch 26/200, Loss: 2.172189474105835
E

In [12]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.4413
